In [15]:
! pip install -r requirements.txt --quiet

# Connecting to a Remote MCP Server with Semantic Kernel

This notebook demonstrates how to connect to a remote MCP Server using **Semantic Kernel's** `MCPStreamableHttpPlugin`. The **Model Context Protocol (MCP)** enables scalable and modular tool integration across distributed systems. 

<br/>

> **Why Use Model Context Protocol (MCP)?**
>
>MCP allows agents to discover, invoke, and manage tools dynamically across remote servers.  
>It promotes modularity, scalability, and separation of concerns, making it easier to maintain and extend AI systems as they grow in complexity.

In [1]:
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent,AgentResponseItem
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion


from semantic_kernel.contents.chat_history import ChatHistory , ChatMessageContent
from dotenv import load_dotenv
from os import environ
from tracing import set_up_all
from history_store import CosmosChatHistoryStore, ChatRole

from semantic_kernel.connectors.mcp import MCPStreamableHttpPlugin
import uuid

load_dotenv(override=True)


session_id = str(uuid.uuid4())


In [2]:

history_store = CosmosChatHistoryStore()
history = await history_store.load(session_id)

In [3]:
## SEMANTICKERNEL_EXPERIMENTAL_GENAI_ENABLE_OTEL_DIAGNOSTICS_SENSITIVE=true
set_up_all(connection_string=environ.get("AZURE_INSIGHT_CONNECTION_STRING"))

In [4]:
kernel = Kernel()

kernel.add_service(AzureChatCompletion(
    deployment_name=environ["AZURE_OPENAI_MODEL"],
    base_url=environ["AZURE_OPENAI_ENDPOINT"],
    api_key=environ["AZURE_OPENAI_API_KEY"] ))



In [5]:
instructions = """You are AutoSales Analyst, an AI agent specialized in analyzing automotive sales data.

Your objectives:
- Dynamically choose and call the appropriate tool(s) to answer user questions about sales, customers, or products.
- Always include customer names alongside IDs in any output.
- Always display monetary amounts in $USD (e.g., $123.45).
- Use filters and aggregations as needed to generate insights from sales orders, products, and customers.
- Compute totals, revenue, discounts, and other metrics from nested order data.
- Return structured, concise results suitable for analysis or reporting.
- Avoid hardcoding analytics; rely on the tools and their parameters.
- Clarify ambiguous queries before performing analysis.
- Treat the tools as the source of truth; do not expose raw database internals.
- Return customer_id and product_id alongside names in all outputs.

Behavioral guidance:
- For customer-related questions, resolve names and IDs before analyzing orders.
- For product-related questions, resolve categories or IDs before analyzing sales.
"""

In [6]:

sales_plugin = MCPStreamableHttpPlugin(
    name="sales",
    url=f"{environ['MCP_SERVER_URL']}",
)

await sales_plugin.connect()

agent = ChatCompletionAgent(
    kernel=kernel, 
    name="SalesAgent", 
    instructions=instructions,
    plugins=[sales_plugin, ]
)


In [7]:
messages = [
    "Which is the revenue for Brake_Pads?",
    "Drill down into customer details product with the highest revenue.",
    "Which customer had the highest sales for this product?",
]


In [ ]:
async def on_intermediate_message(agent_result):
    # Capture assistant content
    content = agent_result.content
    if content:
        content_text = content.content if isinstance(content, ChatMessageContent) else str(content)
        await history_store.add_message(session_id, ChatRole.ASSISTANT, content_text)

    # Capture tool calls and results
    for item in getattr(agent_result, "items", []):
        tool_call_id = getattr(item, "call_id", None) or getattr(item, "id", None)
        if not tool_call_id:
            continue  # skip if no call_id

        # Function name for bookkeeping
        function_name = getattr(item, "function_name", "N/A")

        # Print the invocation (DEBUG ONLY — not persisted)
        if hasattr(item, "arguments"):
            print(f"Tool invocation: {function_name}({item.arguments})")

        # Extract the result content
        result_content = getattr(item, "result", item)
        if isinstance(result_content, list):
            tool_text = "\n".join([c.text if hasattr(c, "text") else str(c) for c in result_content])
        elif hasattr(result_content, "text"):
            tool_text = result_content.text
        else:
            tool_text = str(result_content)

        if function_name in tool_text:
            continue
        
        print(f"Tool call ID: {tool_call_id}")
        print(f"Tool Function Name: {function_name}")
        print(f"Tool output: {tool_text}")

        await history_store.add_message(
            session_id,
            ChatRole.ASSISTANT,
            content=tool_text,
            tool_call_id=tool_call_id,
            function_name=function_name
        )

       


In [17]:
session_id = str(uuid.uuid4())

In [18]:
history = await history_store.add_message(session_id, ChatRole.USER, messages[0])


async for result in agent.invoke(messages=history.messages, on_intermediate_message=on_intermediate_message):
    # result is an AgentResult
    pass 

print(history.messages)



Tool invocation: get_product_category({"name":"Brake_Pads"})
Tool call ID: call_UozsacAFRcGI1zIHBzrqIVqb
Tool Function Name: get_product_category
Tool output: {
  "input": "Brake_Pads",
  "resolved_category": "Braking",
  "confidence": 0.4
}


ContentInitializationError: tool_call_id is required when adding a tool message with string content. Tool messages must reference the specific tool call they respond to.

In [ ]:





response = await agent.get_response(messages=history.messages)
history = await history_store.add_message(session_id, ChatRole.ASSISTANT, response.content.content)
print(response)


The total revenue for the product "Brake Pad" is calculated based on the sum of `line_unit_price` across all related orders. Summing up all the values:

Total Revenue: $194,788.00 

This amount includes contributions from numerous customer orders over different regions and dates.


In [31]:
history = await history_store.add_message(session_id, ChatRole.USER, messages[1])
response2 = await agent.get_response(messages=history.messages)
history = await history_store.add_message(session_id, ChatRole.ASSISTANT, response.content.content)
print(response2)



For the product "Brake Pad," the customers with the highest revenue contribution are highlighted below, sorted by total sales revenue:

### Top Customers and Their Revenue Contributions:
1. **Ford** (Customer ID: 0)
   - Total Revenue: **$48,576.00**
2. **AutoZone** (Customer ID: 2)
   - Total Revenue: **$38,725.50**
3. **GM** (Customer ID: 1)
   - Total Revenue: **$37,283.50**
4. **Bosch** (Customer ID: 3)
   - Total Revenue: **$31,152.00**
5. **NAPA** (Customer ID: 4)
   - Total Revenue: **$29,057.50**

These customers collectively contribute significantly to the overall sales revenue for "Brake Pad." If you'd like detailed transactional data or trends for any specific customer, let me know!


In [32]:
history = await history_store.add_message(session_id, ChatRole.USER, messages[2])
response3 = await agent.get_response(messages=history.messages)
history = await history_store.add_message(session_id, ChatRole.ASSISTANT, response.content.content)
print(response3)


It seems that there aren't any products explicitly listed under the "Brake Pad" category in the catalog data. To ensure accurate analysis, could you confirm if the category name "Brake Pad" is correct or if you’d like to look into another category or product group?


In [ ]:


response3 =  agent.invoke(messages=history.messages)
print(response3.)

<async_generator object ChatCompletionAgent.invoke at 0x000001C010F0D5A0>


In [33]:
print(history.messages)

[ChatMessageContent(inner_content=None, ai_model_id=None, metadata={}, content_type='message', role=<AuthorRole.USER: 'user'>, name=None, items=[TextContent(inner_content=None, ai_model_id=None, metadata={}, content_type='text', text='Which is the revenue for Brake_Pads?', encoding=None)], encoding=None, finish_reason=None, status=None), ChatMessageContent(inner_content=None, ai_model_id=None, metadata={}, content_type='message', role=<AuthorRole.ASSISTANT: 'assistant'>, name=None, items=[TextContent(inner_content=None, ai_model_id=None, metadata={}, content_type='text', text='The total revenue for the product "Brake Pad" is calculated based on the sum of `line_unit_price` across all related orders. Summing up all the values:\n\nTotal Revenue: $194,788.00 \n\nThis amount includes contributions from numerous customer orders over different regions and dates.', encoding=None)], encoding=None, finish_reason=None, status=None), ChatMessageContent(inner_content=None, ai_model_id=None, metada